In [1]:
import sys
sys.path.append("../common_code/")
import common as com

The Symbi-OS project offers a low level mechanism called Elevation, which allows users to execute a system call and toggle the hardware mode of execution, something that is normally forbidden by the operating system. Once elevated, application code can run in supervisor mode, allowing modification to the previously guarded OS structures. 

The elevation mechanism is low level and "close to the hardware". What we mean by this is that there are not software dependencies that restrict which languages or frameworks can be used with this tool. We're simply executing machine code in an alternative mode of execution. This mode allows us to execute a superset of instructions that can be run in usermode. Some modifications have to be made for a number of reasons: 1. Application code and kernel code may be built slightly differently so that there are challenges when unifying their code paths, e.g. application code on X86_64 may use the red zone, but kernel code does not. 2. kernel software safety checks 3. conditional hardware paths such as performing a stack switch based on execution mode.

In these examples, we demonstrate that it is possible to use the elevation mechanism in order to execute a privileged instruction, reading the CR3 system register. It is only possible to access this register from supervisor mode.

# C++

C++ is very close to C, so there's not much of a surprise that it can be used with the elevation mechanism. We can use it to launch an application in the elevated mode using the scripts we designed for use with C programs.

In [2]:
# NYI, need to implement cpp read cr3

# Rust

Rust is a compiled language built for speed, memory & thread safety. It's a systems language with it's own inline assembly and compiler. While most of our examples make use of GCC, the rust compiler is a frontend for LLVM, demonstrating the use of the elevation mechanism is orthogonal to the compiler used.

Here, we show rust source code that reads the CR3 register.

```rust
use std::arch::asm;
fn bar() -> u64 {
    let mut x: u64;
    unsafe {
        asm!("mov {}, cr3", out(reg) x);
    }
    return x;
}

fn main() {
    let cr3 = bar();
    println!("CR3: {:x}", cr3);
}
```
Next we build the code.

In [2]:
# Build rust code
make_rust = "make -C " + com.symbios_path + "LinuxPrototypes/rust/print_cr3/"
ret = com.run_cmd(make_rust + " clean")
print(ret.stdout)
ret = com.run_cmd(make_rust)
print(ret.stdout)

make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'
rm -f main
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'

make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'
rustc -O main.rs
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'



Here we run the code without elevation. This is designed to fail because reading the CR3 register is a privileged instruction that should not be reachable from usermode.

In [3]:
# UNIT TEST: Expected to fail. Expect to see Segmentation fault.
ret = com.run_cmd(make_rust + " run")
print(ret.stdout)
print(ret.stderr)


error:  2
make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'
./main
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'

make: *** [Makefile:4: run] Segmentation fault (core dumped)



This time we run the code with elevation. This time it succeeds. Here "be" means "begin elevated"

In [5]:
# com.add_sc_to_path()

ret = com.run_cmd(make_rust + " be")
print(ret.stdout)
print(ret.stderr)


error:  2
make: Entering directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'
shortcut.sh -be --- ./main
make: Leaving directory '/home/sym/Symbi-OS/LinuxPrototypes/rust/print_cr3'

make: shortcut.sh: No such file or directory
make: *** [Makefile:9: be] Error 127



In [2]:
import os
print(os.environ["PATH"])

/usr/bin:/home/sym/.local/bin:/home/sym/.vscode-server/bin/ee2b180d582a7f601fa6ecfdad8d9fd269ab1884/bin/remote-cli:/home/sym/.local/bin:/home/sym/bin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/sym/Tools/bin:/home/sym/Tools/bin/recipes:/home/sym/Tools/bin/shortcut:/home/sym/.vscode-server/bin/ee2b180d582a7f601fa6ecfdad8d9fd269ab1884/bin/remote-cli:/home/sym/.local/bin:/home/sym/bin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/sym/Tools/bin:/home/sym/Tools/bin/recipes:/home/sym/Tools/bin/shortcut
